In [ ]:
# result hf files
mistral_cc_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_0.pickle"
mistral_orbis_1 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_orbis_1.pickle"
CatPPT_orbis_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/CatPPT_orbis_0.pickle"
mistral_cc_1 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_cc_1.pickle"
mistral_cc_2 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_cc_2.pickle"
CatPPT_cc_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/CatPPT_cc_0.pickle"

# gpt files

gpt35_orbis_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/run_gpt35-turbo_orbis_0.pickle"
gpt35_cc_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/gpt35_turbo_cc_0.pickle"
gpt4_orbis_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/gpt4_orbis_0.pickle"
gpt35_inst_orbis_0 = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/gpt35_instruct_orbis_0.pickle"

# Evaluate the LLMs performance

In [ ]:
# function that just prints results
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score


def evaluate(outcome_file):
    df = pd.read_pickle(outcome_file)
    # Ensure all entries in 'class' and 'predicted' are lists
    df["class"] = df["class"].apply(lambda x: x if isinstance(x, list) else [])
    df["predicted"] = df["predicted"].apply(lambda x: x if isinstance(x, list) else [])

    # Initialize MultiLabelBinarizer
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on the 'class' column only, assuming 'class' and 'predicted' have the same classes
    mlb.fit(df["class"])

    # Transform 'class' and 'predicted' columns into binary format
    y_true = mlb.transform(df["class"])
    y_pred = mlb.transform(df["predicted"])

    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred, average="micro")
    recall = recall_score(y_true, y_pred, average="micro")
    f1 = f1_score(y_true, y_pred, average="micro")

    # Print the scores
    print(f"Precision: {round(precision, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"F1 Score: {round(f1, 2)}")

    # Count the number of exact matches
    exact_matches = sum(
        [
            set(actual) == set(predicted)
            for actual, predicted in zip(df["class"], df["predicted"])
        ]
    )

    # Calculate the exact match ratio by dividing the number of exact matches by the total number of instances
    exact_match_ratio = exact_matches / len(df)

    print(f"Exact matches: {exact_matches} out of {len(df)}")
    print(f"Exact Match Ratio: {round(exact_match_ratio, 2)}")

    # Count the number of partial matches (at least one label is correct)
    partial_matches = sum(
        [
            len(set(actual) & set(predicted)) > 0
            for actual, predicted in zip(df["class"], df["predicted"])
        ]
    )

    # Calculate the partial match ratio by dividing the number of partial matches by the total number of instances
    partial_match_ratio = partial_matches / len(df)
    print(f"Partial Matches: {partial_matches} out of {len(df)}")
    print(f"Partial Match Ratio: {round(partial_match_ratio, 2)}")

In [ ]:
evaluate(
    "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/test_gollie_0.pickle"
)

In [ ]:
evaluate(
    "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_cc_regex.pickle"
)

In [ ]:
evaluate(
    "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/mistral_cc_2.pickle"
)

In [ ]:
evaluate(
    "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/gollie_orbis_test2"
)

In [ ]:
evaluate(
    "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/gollie_cc_0.pickle"
)

In [ ]:
# function that returns a result dataframe
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score


def evaluate(outcome_file):
    df = pd.read_pickle(outcome_file)
    df["class"] = df["class"].apply(lambda x: x if isinstance(x, list) else [])
    df["predicted"] = df["predicted"].apply(lambda x: x if isinstance(x, list) else [])

    mlb = MultiLabelBinarizer()
    mlb.fit(df["class"])
    y_true = mlb.transform(df["class"])
    y_pred = mlb.transform(df["predicted"])

    precision = precision_score(y_true, y_pred, average="micro")
    recall = recall_score(y_true, y_pred, average="micro")
    f1 = f1_score(y_true, y_pred, average="micro")

    exact_matches = sum(
        [
            set(actual) == set(predicted)
            for actual, predicted in zip(df["class"], df["predicted"])
        ]
    )
    exact_match_ratio = exact_matches / len(df)

    partial_matches = sum(
        [
            len(set(actual) & set(predicted)) > 0
            for actual, predicted in zip(df["class"], df["predicted"])
        ]
    )
    partial_match_ratio = partial_matches / len(df)

    # Creating DataFrame with object type to allow mixed types
    results_df = pd.DataFrame(
        {
            "Metric": [
                "Precision",
                "Recall",
                "F1 Score",
                "Exact Matches",
                "Exact Match Ratio",
                "Partial Matches",
                "Partial Match Ratio",
                "Nr Firms",
            ],
            "Value": [
                round(precision, 2),  # float
                round(recall, 2),  # float
                round(f1, 2),  # float
                exact_matches,  # int, stored as object
                round(exact_match_ratio, 2),  # float
                partial_matches,  # int, stored as object
                round(partial_match_ratio, 2),  # float
                len(df),  # int, stored as object
            ],
        },
        dtype=object,
    )  # Setting dtype as object

    return results_df

## GPT-4

### on orbis

In [ ]:
evaluate(gpt4_orbis_0)

## GPT 3.5-turbo

### on orbis

In [ ]:
evaluate(gpt35_orbis_0)

### on cc

In [ ]:
evaluate(gpt35_cc_0)

## Mistral-7B-inst

### on commoncrawl

In [ ]:
evaluate(mistral_cc_0)

on commoncrawl 2.0 (with enhanced similarity search)

In [ ]:
evaluate(mistral_cc_1)

In [ ]:
evaluate(mistral_cc_2)

## on orbis


In [ ]:
evaluate(mistral_orbis_1)

## CatPPT

model: rishiraj/CatPPT

this is the top model of the [LLM-performance-leaderboard](https://huggingface.co/spaces/optimum/llm-perf-leaderboard) with 7B parameters which is in a memory size (ca. 16GB) that we can use on our GPU

### on orbis

In [ ]:
evaluate(CatPPT_orbis_0)

## How would random predicition go?

In [ ]:
def calculate_expected_results(n_rows, n_classes, prob_predict):
    """
    Calculate and print expected precision, recall, F1 score, exact matches, and partial matches
    for a completely random multi-label classification scenario.

    Parameters:
    - n_rows: int, number of instances (rows) in the dataset.
    - n_classes: int, number of possible classes.
    - prob_predict: float, probability that a class is predicted for an instance.
    """
    # Calculations
    expected_precision = prob_predict
    expected_recall = prob_predict  # Assuming the probability of a relevant label being selected is the same
    expected_f1 = (
        2
        * (expected_precision * expected_recall)
        / (expected_precision + expected_recall)
    )

    # Exact Match Ratio: Probability of predicting the exact set of labels correctly by chance
    exact_match_ratio = prob_predict**n_classes

    # Partial Match Ratio: Probability of getting at least one label correct
    partial_match_ratio = 1 - ((1 - prob_predict) ** n_classes)

    # Results for N rows
    expected_exact_matches = n_rows * exact_match_ratio
    expected_partial_matches = n_rows * partial_match_ratio

    # Print the expected results
    print(f"Precision: {expected_precision}")
    print(f"Recall: {expected_recall}")
    print(f"F1 Score: {expected_f1}")
    print(f"Exact Matches: Approximately {expected_exact_matches} instances")
    print(f"Partial Matches: Approximately {expected_partial_matches} instances")

In [ ]:
# With 116 rows, 5 classes, and 0.5 probability of predicting a class
calculate_expected_results(116, 5, 0.5)

In [ ]:
# With 116 rows, 5 classes, and 0.2 probability of predicting a class
calculate_expected_results(116, 5, 0.2)

In [ ]:
# if we took the 80 classes from gt:

calculate_expected_results(116, 80, 0.02)

## Make result table

In [ ]:
### Final table
# Define the base path
base_path = "/home/zelle/development/projects/ascii/reference-data/data_raw_direct_source_drop/joshua/llm_data/"

# Define file paths relative to the base path
file_paths = {
    "hf_models": [
        base_path + "mistral_0.pickle",
        base_path + "mistral_orbis_1.pickle",
        base_path + "CatPPT_orbis_0.pickle",
        # base_path + 'mistral_cc_1.pickle',
        base_path + "mistral_cc_2.pickle",
        base_path + "CatPPT_cc_0.pickle",
        base_path + "gollie_cc_0.pickle",
        base_path + "gollie_orbis_0.pickle",
    ],
    "openai_models": [
        base_path + "run_gpt35-turbo_orbis_0.pickle",
        base_path + "gpt35_turbo_cc_0.pickle",
        base_path + "gpt4_orbis_0.pickle",
        # base_path + 'gpt35_instruct_orbis_0.pickle'
    ],
}

In [ ]:
# Initialize an empty DataFrame for concatenated results
concatenated_results = pd.DataFrame()

# Iterate over all files, evaluate, and concatenate
for category, files in file_paths.items():
    for file in files:
        file_path = file
        result_df = evaluate(file_path)

        # Rename 'Value' column to the file name without '.pickle' extension
        result_df.rename(
            columns={"Value": file.replace(".pickle", "").replace(base_path, "")},
            inplace=True,
        )

        # Concatenate the results
        if concatenated_results.empty:
            concatenated_results = result_df
        else:
            concatenated_results = pd.concat(
                [concatenated_results, result_df.iloc[:, 1]], axis=1
            )

# Now concatenated_results contains all the data

In [ ]:
concatenated_results

In [ ]:
# Initialize two empty DataFrames for cc and orbis results
cc_results = pd.DataFrame()
orbis_results = pd.DataFrame()

# Iterate over all files, evaluate, and concatenate based on filename
for category, files in file_paths.items():
    for file in files:
        file_path = file
        result_df = evaluate(file_path)

        # Check if 'cc' is in the filename and concatenate to cc_results
        if "cc" in file:
            result_df.rename(
                columns={"Value": file.replace(".pickle", "").replace(base_path, "")},
                inplace=True,
            )
            if cc_results.empty:
                cc_results = result_df
            else:
                cc_results = pd.concat([cc_results, result_df.iloc[:, 1]], axis=1)

        # Check if 'orbis' is in the filename and concatenate to orbis_results
        elif "orbis" in file:
            result_df.rename(
                columns={"Value": file.replace(".pickle", "").replace(base_path, "")},
                inplace=True,
            )
            if orbis_results.empty:
                orbis_results = result_df
            else:
                orbis_results = pd.concat([orbis_results, result_df.iloc[:, 1]], axis=1)

# cc_results now contains all data for 'cc' files
# orbis_results now contains all data for 'orbis' files

In [ ]:
cc_results

In [ ]:
orbis_results

In [ ]:
# Example to pivot the cc_results DataFrame
cc_results_pivoted = cc_results.set_index("Metric").transpose().reset_index()
cc_results_pivoted.rename(columns={"index": "Model"}, inplace=True)

In [ ]:
latex_orbis = orbis_results.to_latex(
    index=False,
    caption="Model Evaluation Metrics on Orbis",
    label="tab:orbis_metrics",
    float_format="%.2f",
)

In [ ]:
print(latex_orbis)

In [ ]:
latex_cc = cc_results.to_latex(
    index=False,
    caption="Model Evaluation Metrics on CommonCrawl",
    label="tab:cc_metrics",
    float_format="%.2f",
)
print(latex_cc)